<a href="https://colab.research.google.com/github/ljkrajewski/jupyter_notebooks/blob/main/flux/GUFlux_mm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Prompts and variables
prompts = [
"Black and white, line drawing. A dimly lit apartment bedroom at night, where a person sits on the edge of their bed with headphones on, eyes closed and expression tense. Wisps of shadowy figures, semi-transparent but distinct, emerge from the walls and corners, each with unique faces and expressions—some whispering, some shouting, others reaching out. Floating text messages and glowing waveforms swirl around them, symbolizing the bombardment of voices. The words ""Not all voices are imaginary."" are glowing faintly on a poster taped to the wall, its letters flickering subtly like a faulty neon sign.",
"Black and white, line drawing. A crowded subway car, where a person stands gripping a pole, eyes darting nervously. Around them, other passengers blur and glitch, their faces shifting between familiar and monstrous. Speech bubbles in different fonts and colors float overhead, filled with fragmented sentences, confessions, and accusations. The train windows reflect a different scene entirely—an empty car with the same person sitting alone. The words ""Not all voices are imaginary."" are graffitied on the metal wall beside a door, some letters dripping like fresh paint.",
"Black and white, line drawing. A therapist's office with warm lighting and soft furnishings, but with an unsettling twist: multiple versions of the same person sit in different chairs, each with distinct body language and expressions—anger, sorrow, fear, and calm. The actual person sits on the main couch, head in hands, overwhelmed. A framed diploma on the wall reads ""Not all voices are imaginary."" in elegant cursive, the letters slightly warped and out of focus. A tape recorder on the table shows sound waves pulsing, hinting that some voices are very real.",
"Black and white, line drawing. A dense forest at dusk, where trees seem to lean in, their branches twisted like grasping hands. A person stands on a narrow path, lantern in hand, with a fearful expression. Whispering faces form in the mist, eyes glowing faintly, while one tree has a face carved into it, eyes half-lidded and mouth open as if speaking. A wooden sign nailed to a tree reads: ""Not all voices are imaginary."" with moss creeping over the edges, letters rough and uneven. Fireflies float around, their glow casting eerie highlights.",
"Black and white, line drawing. A gothic cathedral interior, stained glass windows casting fractured light onto the marble floor. The pews are empty, but shadowy figures in faint priestly robes linger near the altar, mouths moving silently. A lone person stands in the center aisle, clutching their head as whispers echo from every direction. A banner hanging above the altar displays the words ""Not all voices are imaginary."" in intricate, blood-red calligraphy. Candles flicker unevenly, casting long, shifting shadows."
]
target = "0016-022"
quantity = 6
wh_ratio = "3:2"
orientation = "landscape"
steps = 20
guidance = 3.5
lora_strength_model = 1.0
lora_strength_clip = 1.0
sampler_name = "euler"
scheduler = "simple"
initseed = 0

In [2]:
#@title _Code be in this here box_
%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2 gradio==3.50.2 python-multipart==0.0.12
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors -d /content/TotoroUI/models/checkpoints -o flux1-dev-fp8-all-in-one.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux_realism_lora.safetensors -d /content/TotoroUI/models/loras -o flux_realism_lora.safetensors

import math
import random
import torch
import numpy as np
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
import gradio as gr
import os

#def add_ai_metadata(image_path,seed,prompt):
#  """
#  Adds AI generation metadata (seed and prompt) to an image.
#
#  Args:
#      image_path (str): Path to the image file.
#      seed (int): Random seed used for the image generation.
#      prompt (str): Text prompt used for the image generation.
#
#  Returns:
#      None
#  """
#  with Image.open(image_path) as img:
#    metadata = img.info
#    metadata["seed"] = str(seed)
#    metadata["prompt"] = prompt
#    img.save(image_path, **metadata)
#    print(f"AI generation metadata added to {image_path}")

def add_ai_metadata(image_path, metadata):
  """Adds metadata to a PNG image.

  Args:
    image_path: Path to the image file.
    metadata: A dictionary containing key-value pairs of metadata.
  """

  targetImage = Image.open("pathToImage.png")
  metadata = PngInfo()
  metadata.add_text("MyNewString", "A string")
  metadata.add_text("MyNewInt", str(1234))

  targetImage.save("NewPath.png", pnginfo=metadata)
  print(f"AI generation metadata added to {image_path}")

def optimal_dimensions(wh_ratio):
    """
    Calculates optimal dimensions for Stable Diffusion.

    Args:
      wh_ratio (str): Width-to-height ratio in the format "width:height".

    Returns:
      A tuple of (new_width, new_height) representing the optimal dimensions.
    """
    sw, sh = wh_ratio.split(':')
    w, h = int(sw), int(sh)
    c = math.sqrt(1024**2 / (w * h))
    new_width = int(((w * c) // 16) * 16)
    new_height = int(((h * c) // 16) * 16)
    #print(f"Optimal dimensions: {new_width}x{new_height}")
    return new_width, new_height

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet, clip, vae = CheckpointLoaderSimple.load_checkpoint("flux1-dev-fp8-all-in-one.safetensors")

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
#def generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
def generate(positive_prompt, wh_ratio, orientation, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    global unet, clip
    if initseed == 0:
        seed = random.randint(0, 18446744073709551615)
    else:
        seed = initseed
    print(f"\nSeed:  {seed}")
    #print(f"Prompt:  {positive_prompt}")
    width, height = optimal_dimensions(wh_ratio)
    if orientation == "portrait":
        width, height = height, width
    print(f"Dimentions:  {width}x{height} ({orientation})")
    #print(f"Width:  {width}\tType:  {type(width)}")
    #print(f"Height:  {height}\tType:  {type(height)}")
    #print(f"Steps:  {steps}\tType:  {type(steps)}
    unet_lora, clip_lora = LoraLoader.load_lora(unet, clip, "flux_realism_lora.safetensors", lora_strength_model, lora_strength_clip)
    cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_lora, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")
    metadata = {
        "prompt": positive_prompt,
        "seed": seed,
        "sampler": sampler_name,
        "scheduler": scheduler,
        "guidance": guidance,
        "lora_strength_model": lora_strength_model,
        "lora_strength_clip": lora_strength_clip
    }
    #add_ai_metadata("/content/flux.png", metadata)
    return "/content/flux.png"

def generate_wrapper(prompts, quantity, wh_ratio, orientation, initseed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    for i in range(len(prompts)):
      for letter in [chr(ord('a') + i) for i in range(quantity)]:
          print(f"----\nGenerating image {i+1}{letter}.png...")
          generate(prompts[i], wh_ratio, orientation, initseed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip)
          os.rename("/content/flux.png", f"/content/{i+1}{letter}.png")
    return f"/content/{i+1}{letter}.png"

"""
with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            positive_prompt = gr.Textbox(lines=3, interactive=True, value="Anime drawing, full body portrait, attractive 19-year-old Caucasian woman, long straight blonde hair, red lipstick, white button-up blouse, black neck tie, black suspenders, tan suit jacket, tan shorts, tan pantyhose, tan flat shoes, smiling, sitting in a recliner, legs crossed", label="Prompt")
            #width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="width")
            #height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="height")
            with gr.Row():
                #wh_ratio = gr.Textbox(lines=1, interactive=True, value="4:7", label="width:height ratio")
                #wh_ratio = gr.Dropdown(["1:1","2:1","3:2","5:3","7:4","9:7","16:9","21:11","17:15"], value="7:4", label="width:height ratio")
                #orientation = gr.Dropdown(["portrait", "landscape"], label="orientation", value="portrait")
                #quantity = gr.Slider(minimum=1, maximum=10, value=1, step=1, label="quantity")
                #width, height = optimal_dimensions(wh_ratio.value)
            #seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="seed (0=random)")
            #steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="steps")
            #guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="guidance")
            #lora_strength_model = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_model")
            #lora_strength_clip = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_clip")
            #sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "heunpp2", "dpm_2", "lms", "dpmpp_2m", "ipndm", "deis", "ddim", "uni_pc", "uni_pc_bh2"], label="sampler_name", value="euler")
            #scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="scheduler", value="simple")
            #generate_button = gr.Button("Generate")
        with gr.Column():
            output_image = gr.Image(label="Generated image", interactive=False)
"""

    #generate_button.click(fn=generate, inputs=[positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)
    #generate_button.click(fn=generate_wrapper, inputs=[positive_prompt, quantity, wh_ratio, orientation, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)
generate_wrapper(prompts, quantity, wh_ratio, orientation, initseed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip)
%cd /content
!tar -cvzf {target}.tar.gz *.png
!rm *.png

#demo.queue().launch(inline=False, share=True, debug=True)

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 14652, done.
remote: Total 14652 (delta 0), reused 0 (delta 0), pack-reused 14652 (from 1)
Receiving objects: 100% (14652/14652), 21.13 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (9824/9824), done.
/content/TotoroUI
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s 

  0%|          | 0/20 [00:00<?, ?it/s]

----
Generating image 1b.png...

Seed:  9332615074622136363
Dimentions:  1248x832 (landscape)


  0%|          | 0/20 [00:00<?, ?it/s]

----
Generating image 1c.png...

Seed:  1523142034341311697
Dimentions:  1248x832 (landscape)


...

----
Generating image 5e.png...

Seed:  14847513579355611893
Dimentions:  1248x832 (landscape)


  0%|          | 0/20 [00:00<?, ?it/s]

----
Generating image 5f.png...

Seed:  6904875773271165332
Dimentions:  1248x832 (landscape)


  0%|          | 0/20 [00:00<?, ?it/s]

tar: Removing leading `/' from member names
/content/1a.png
tar: Removing leading `/' from hard link targets
/content/1b.png
/content/1c.png
/content/1d.png
/content/1e.png
/content/1f.png
/content/2a.png
/content/2b.png
/content/2c.png
/content/2d.png
/content/2e.png
/content/2f.png
/content/3a.png
/content/3b.png
/content/3c.png
/content/3d.png
/content/3e.png
/content/3f.png
/content/4a.png
/content/4b.png
/content/4c.png
/content/4d.png
/content/4e.png
/content/4f.png
/content/5a.png
/content/5b.png
/content/5c.png
/content/5d.png
/content/5e.png
/content/5f.png


In [3]:
%cd /content
!tar -cvzf 0010-043.tar.gz *.png

/content
1a.png
1b.png
1c.png
1d.png
1e.png
1f.png
2a.png
2b.png
2c.png
2d.png
2e.png
2f.png
3a.png
3b.png
3c.png
3d.png
3e.png
3f.png
4a.png
4b.png
4c.png
4d.png
4e.png
4f.png
5a.png
5b.png
5c.png
5d.png
5e.png
5f.png


In [4]:
!rm *.png